In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
#from category_encoders.one_hot import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [6]:
y = df["class"]
y = y.map({"p":-1, "e":1})

In [7]:
X = df.drop(["class"], axis=1)

In [23]:
X.isnull().sum()

id                            0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [25]:
X.shape

(3116945, 21)

In [46]:
y.value_counts()

class
-1    1705396
 1    1411549
Name: count, dtype: int64

In [8]:
X = X.drop(["stem-root", "veil-type", "veil-color", "spore-print-color"], axis=1)

In [48]:
X.shape

(3116945, 17)

In [50]:
X.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season
0,0,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,w,f,f,d,a
1,1,4.51,x,h,o,f,a,c,n,4.79,6.48,y,o,t,z,d,w
2,2,6.94,f,s,b,f,x,c,w,6.85,9.93,s,n,f,f,l,w
3,3,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,w,f,f,d,u
4,4,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,w,f,f,g,a


In [9]:
X = X.drop(["gill-spacing", "stem-surface"], axis=1)

In [53]:
X.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,0,8.80,f,s,u,f,a,w,4.51,15.39,w,f,f,d,a
1,1,4.51,x,h,o,f,a,n,4.79,6.48,o,t,z,d,w
2,2,6.94,f,s,b,f,x,w,6.85,9.93,n,f,f,l,w
3,3,3.88,f,y,g,f,s,g,4.16,6.53,w,f,f,d,u
4,4,5.85,x,l,w,f,d,w,3.37,8.36,w,f,f,g,a


### Выделение и преобразование категориальных признаков

In [77]:
cat_cols = ["cap-shape","cap-surface",  "cap-color", "does-bruise-or-bleed", "gill-attachment","gill-color", "stem-color", "has-ring", "ring-type", "habitat", "season"]

In [11]:
cat_cols = ["cap-shape"]

In [15]:
data_new = OneHotEncoder(dtype = 'int').fit_transform(X.values)
pd.DataFrame(data_new.toarray(),
            columns=onehotencoder.cat_cols).head()

MemoryError: Unable to allocate 35.5 TiB for an array with shape (3116945, 3130034) and data type int32

In [92]:
default_pipeline = Pipeline([
    ('cat_encoder',  LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [93]:
cv_res1 = cross_validate(default_pipeline,
                         X,
                         y,
                         cv=5,
                         scoring='f1',
                         return_train_score=True
                         )

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/malleo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/malleo/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/malleo/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/malleo/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 408,

In [63]:
X["cap-surface"]###, "bruises", "gill-spacing","does-bruise-or-bleed", "gill-attachment", "gill-color", "stem-color", "has-ring", "ring-type", "habitat", "season"]

0            s
1            h
2            s
3            y
4            l
          ... 
3116940    NaN
3116941    NaN
3116942      e
3116943      i
3116944      s
Name: cap-surface, Length: 3116945, dtype: object

In [85]:
default_pipeline 

IndexError: list index out of range

IndexError: list index out of range

Pipeline(steps=[])